In [ ]:
from sepal_ui import aoi
from component.tile import *
from component.model import *

In [ ]:
# Define area of interest tile
aoi_tile = aoi.AoiTile()
aoi_tile

In [ ]:
model = MgciModel()

In [ ]:
mountain_tile = MountainTile(aoi=aoi_tile, model=model)

In [ ]:
mountain_tile

In [ ]:
vegetation_tile = VegetationTile(aoi_model=aoi_tile.view.model)

In [ ]:
vegetation_tile

In [ ]:
from component.scripts.scripts import *

In [ ]:
mountain_layer = mountain_tile.model.kapos_image
vegetation_layer = vegetation_tile.reclassify_tile.model.reclass_ee
asset_type = vegetation_tile.reclassify_tile.model.asset_type
aoi_geometry = aoi_tile.view.model.feature_collection.geometry()

In [ ]:
vegetation_layer

In [ ]:
mountain_layer

In [ ]:
ee.Image.pixelArea().divide(10000)\
    .updateMask(vegetation_layer.mask().And(mountain_layer.mask()))\
    .reduceRegion(reducer=ee.Reducer.sum(), scale=300, geometry=aoi_geometry).getInfo()

In [ ]:
def get_lulc_area_per_class(lulc, kapos, aoi, scale=30):
    
    """Reduce land use/land cover image to kapos regions
    
    Args:
        lulc (ee.Image, categorical): Input image to reduce
        kapos (ee.Image, categorical): Input region
        aoi (ee.FeatureCollection, ee.Geometry): Region to reduce image
        scale (int, optional): By default using 30meters as scale
        
    Return:
        ee.Dicionary.
    """

    return ee.Image.pixelArea().divide(10000)\
      .updateMask(lulc.mask().And(kapos.mask()))\
      .addBands(lulc)\
      .addBands(kapos)\
      .reduceRegion(**{
        'reducer': ee.Reducer.sum().group(1).group(2), 
        'geometry': aoi, 
        'maxPixels': 1e13,
        'scale': scale,
        'bestEffort':True,
        'tileScale':4
      })

In [ ]:
result = get_lulc_area_per_class(vegetation_layer, mountain_layer, aoi_geometry, scale=300).getInfo()

In [ ]:
result

In [ ]:
mountain_classes = [result['groups'][i]['group'] for i in range(len(result['groups']))]
mountain_classes

In [ ]:
total_area = 0
for kapos_class in class_area_per_kapos.values():
    for area in kapos_class.values():
        total_area += area

In [ ]:
total_area

In [ ]:
class_area_per_kapos

In [ ]:
mountainClasses = [1, 2, 3, 4, 5]

In [ ]:
def get_area(classification):
    
    area = ee.Image.pixelArea()
        .updateMask(classification)  
        .addBands(classification)
        .reduceRegion(
          reducer=ee.Reducer.sum().group(1), 
          geometry=geometry, 
          scale=30
    )
    
    return area

In [ ]:
def mountain_class(mountain_class):
  var area = getArea(
    classification.updateMask(mountains.eq(mountainClass))
  )
  return {mountainClass: mountainClass, area: area}

In [ ]:
var mountainClasses = [1, 2, 3, 4, 5]
var areaByMountainClass = mountainClasses.map(function (mountainClass) {
  var area = getArea(
    classification.updateMask(mountains.eq(mountainClass))
  )
  return {mountainClass: mountainClass, area: area}
})
print(areaByMountainClass)


function getArea(classification) {
  var area = ee.Image.pixelArea()
    .updateMask(classification)  
    .addBands(classification)
    .reduceRegion({
      reducer: ee.Reducer.sum().group(1), 
      geometry: geometry, 
      scale: 100
    })
    
    return area
}  
  Map.addLayer(classification.randomVisualizer())

In [ ]:
mountain_layer.get('properties').getInfo()

In [ ]:
import geemap

In [ ]:
from component.parameter.directory import *
from pathlib import Path

In [ ]:
# Mountain layer has to be transformed into vector


# Vegetation layer has to be an Image

In [ ]:
out_path = RESULTS_DIR/'zonal_statistics'
out_path.mkdir(parents=True, exist_ok=True)
out_file = out_path/'zonal_{}.csv'.format(aoi_tile.view.model.name)

In [ ]:
out_file